In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, avg, sum as spark_sum, max as spark_max, min as spark_min,
    current_timestamp, lit, when, round as spark_round, to_date, weekofyear, year,
    row_number, percentile_approx, collect_list, struct, countDistinct
)
from pyspark.sql.types import IntegerType, StringType, TimestampType, DateType, FloatType, DecimalType
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F

In [0]:
catalog_name = "atendimento_catalog"
silver_db_name = "silver"
gold_db_name = "gold"

In [0]:
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {gold_db_name}")

## Funções Utilitárias

Funções auxiliares para validação e transformação de dados.

In [0]:
def safe_table_exists(full_name: str) -> bool:
    try:
        spark.table(full_name)
        return True
    except AnalysisException:
        return False
    except Exception:
        return False

def safe_col(df, name):
    return col(name) if name in df.columns else lit(None).cast(StringType())

## Processamento: funnel_metrics

Transformação da camada Silver para Gold com agregações de métricas de funil.
Cálculo de conversões entre etapas e custos operacionais por canal.

### Leitura das Tabelas Silver

Carregamento das tabelas necessárias para métricas de funil.

In [0]:
src_table_chamados = f"{catalog_name}.{silver_db_name}.ft_chamados"
src_table_clientes = f"{catalog_name}.{silver_db_name}.dm_clientes"
src_table_canais = f"{catalog_name}.{silver_db_name}.dm_canais"
src_table_motivos = f"{catalog_name}.{silver_db_name}.dm_motivos"
src_table_atendentes = f"{catalog_name}.{silver_db_name}.ft_atendentes"
src_table_custos = f"{catalog_name}.{silver_db_name}.ft_custos"
tgt_table_funnel = f"{catalog_name}.{gold_db_name}.funnel_metrics"

if not safe_table_exists(src_table_chamados):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_chamados}")
if not safe_table_exists(src_table_clientes):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_clientes}")
if not safe_table_exists(src_table_canais):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_canais}")
if not safe_table_exists(src_table_motivos):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_motivos}")

df_chamados_src = spark.table(src_table_chamados)
df_clientes_src = spark.table(src_table_clientes)
df_canais_src = spark.table(src_table_canais)
df_motivos_src = spark.table(src_table_motivos)

total_before_chamados = df_chamados_src.count()

print(f"Leitura: {src_table_chamados}")
print(f"Registros ft_chamados: {total_before_chamados:,}")

### Join com Dimensões

Enriquecimento com dados de clientes, canais e motivos.

In [0]:
df_base = df_chamados_src.alias("ch") \
    .join(df_clientes_src.alias("cl"), col("ch.id_cliente") == col("cl.id_cliente"), "left") \
    .join(df_canais_src.alias("ca"), col("ch.id_canal") == col("ca.id_canal"), "left") \
    .join(df_motivos_src.alias("m"), col("ch.id_motivo") == col("m.id_motivo"), "left") \
    .select(
        col("ch.id_chamado"),
        col("ch.id_cliente"),
        col("ch.id_atendente"),
        col("ch.id_motivo"),
        col("ch.hora_abertura_chamado"),
        col("ch.hora_inicio_atendimento"),
        col("ch.hora_finalizacao_atendimento"),
        col("ch.tempo_espera_minutos"),
        col("ch.tempo_atendimento_minutos"),
        col("ch.resolvido"),
        col("ca.nome_canal").alias("canal"),
        col("cl.regiao"),
        col("m.nome_motivo").alias("motivo")
    )

if safe_table_exists(src_table_atendentes):
    df_atendentes_src = spark.table(src_table_atendentes)
    df_base = df_base.alias("b") \
        .join(df_atendentes_src.alias("a"), col("b.id_atendente") == col("a.id_atendente"), "left") \
        .select("b.*", col("a.nome_atendente"), col("a.nivel_atendimento"))
else:
    df_base = df_base.withColumn("nome_atendente", lit(None).cast(StringType()))
    df_base = df_base.withColumn("nivel_atendimento", lit(None).cast(IntegerType()))

if safe_table_exists(src_table_custos):
    df_custos_src = spark.table(src_table_custos)
    df_base = df_base.alias("b") \
        .join(df_custos_src.alias("cu"), col("b.id_chamado") == col("cu.id_chamado"), "left") \
        .select("b.*", col("cu.custo"))
else:
    df_base = df_base.withColumn("custo", lit(None).cast(FloatType()))

print(f"Registros após joins: {df_base.count():,}")

### Classificação de Etapas do Funil

Categorização de cada chamado em etapas do funil baseado em canal e nível de atendimento.

In [0]:
df_funnel = df_base.withColumn("data", to_date(col("hora_abertura_chamado")))
df_funnel = df_funnel.withColumn("semana", weekofyear(col("hora_abertura_chamado")))
df_funnel = df_funnel.withColumn("ano", year(col("hora_abertura_chamado")))

df_funnel = df_funnel.withColumn("n_prevencao", lit(0).cast(IntegerType()))

df_funnel = df_funnel.withColumn(
    "n_autosservico",
    when(
        (col("canal").isin(["Ura", "Chatbot", "App"])) & 
        (col("id_atendente").isNull()) &
        (col("hora_finalizacao_atendimento").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_nivel1",
    when(
        (col("nivel_atendimento") == 1) & (col("id_atendente").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_nivel2",
    when(
        (col("nivel_atendimento") == 2) & (col("id_atendente").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_resolvido",
    when(col("resolvido") == "Sim", 1).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "flag_missing_timestamps",
    when(
        (col("hora_abertura_chamado").isNull()) | (col("hora_finalizacao_atendimento").isNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "flag_escalacao_incorreta",
    when(
        (col("nivel_atendimento").isNotNull()) &
        (col("n_autosservico") == 0) &
        (col("canal").isin(["Ura", "Chatbot", "App"])),
        1
    ).otherwise(0).cast(IntegerType())
)

w_cliente = Window.partitionBy("id_cliente", "data").orderBy(col("hora_abertura_chamado"))

df_funnel = df_funnel.withColumn(
    "chamado_seq",
    row_number().over(w_cliente)
)

df_funnel = df_funnel.withColumn(
    "flag_reabertura",
    when(col("chamado_seq") > 1, 1).otherwise(0).cast(IntegerType())
)

print(f"Registros classificados: {df_funnel.count():,}")

### Agregação de Métricas

Agregação de métricas por data e canal com contadores de cada etapa do funil.

In [0]:
df_funnel_agg = df_funnel.groupBy("data", "semana", "ano", "canal", "regiao").agg(
    spark_sum("n_prevencao").alias("n_prevencao"),
    spark_sum("n_autosservico").alias("n_autosservico"),
    spark_sum("n_nivel1").alias("n_nivel1"),
    spark_sum("n_nivel2").alias("n_nivel2"),
    spark_sum("n_resolvido").alias("n_resolvido"),
    count("id_chamado").alias("volume_total_chamados"),
    spark_round(avg("tempo_espera_minutos"), 2).cast(DecimalType(10, 2)).alias("tempo_espera_medio"),
    spark_round(avg("tempo_atendimento_minutos"), 2).cast(DecimalType(10, 2)).alias("tempo_atendimento_medio"),
    spark_sum("flag_missing_timestamps").alias("count_missing_timestamps"),
    spark_sum("flag_reabertura").alias("count_reabertura"),
    spark_sum("flag_escalacao_incorreta").alias("count_escalacao_incorreta"),
    spark_round(spark_sum("custo"), 2).cast(DecimalType(10, 2)).alias("custo_total")
)

df_funnel_agg = df_funnel_agg.withColumn(
    "taxa_conversao_autosservico_nivel1",
    spark_round(
        when(
            col("n_autosservico") > 0,
            (col("n_nivel1") / col("n_autosservico")) * 100
        ).otherwise(lit(0.0)),
        2
    ).cast(DecimalType(10, 2))
)

df_funnel_agg = df_funnel_agg.withColumn(
    "taxa_conversao_nivel1_nivel2",
    spark_round(
        when(
            col("n_nivel1") > 0,
            (col("n_nivel2") / col("n_nivel1")) * 100
        ).otherwise(lit(0.0)),
        2
    ).cast(DecimalType(10, 2))
)

df_funnel_agg = df_funnel_agg.withColumn(
    "taxa_conversao_autosservico_resolvido",
    spark_round(
        when(
            col("n_autosservico") > 0,
            (col("n_resolvido") / col("n_autosservico")) * 100
        ).otherwise(lit(0.0)),
        2
    ).cast(DecimalType(10, 2))
)

df_funnel_agg = df_funnel_agg.withColumn(
    "pct_missing_timestamps",
    spark_round(
        when(
            col("volume_total_chamados") > 0,
            (col("count_missing_timestamps") / col("volume_total_chamados")) * 100
        ).otherwise(lit(0.0)),
        2
    ).cast(DecimalType(10, 2))
)

df_funnel_agg = df_funnel_agg.withColumn(
    "taxa_reabertura",
    spark_round(
        when(
            col("volume_total_chamados") > 0,
            (col("count_reabertura") / col("volume_total_chamados")) * 100
        ).otherwise(lit(0.0)),
        2
    ).cast(DecimalType(10, 2))
)

df_funnel_agg = df_funnel_agg.withColumn(
    "pct_escalacao_incorreta",
    spark_round(
        when(
            col("volume_total_chamados") > 0,
            (col("count_escalacao_incorreta") / col("volume_total_chamados")) * 100
        ).otherwise(lit(0.0)),
        2
    ).cast(DecimalType(10, 2))
)

df_funnel_agg = df_funnel_agg.withColumn("processed_timestamp", current_timestamp())

print(f"Registros agregados: {df_funnel_agg.count():,}")

### Tipagem de Colunas

Aplicação de tipos explícitos às colunas para garantir schema consistente.

In [0]:
df_funnel_typed = df_funnel_agg \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("n_prevencao", col("n_prevencao").cast(IntegerType())) \
    .withColumn("n_autosservico", col("n_autosservico").cast(IntegerType())) \
    .withColumn("n_nivel1", col("n_nivel1").cast(IntegerType())) \
    .withColumn("n_nivel2", col("n_nivel2").cast(IntegerType())) \
    .withColumn("n_resolvido", col("n_resolvido").cast(IntegerType())) \
    .withColumn("volume_total_chamados", col("volume_total_chamados").cast(IntegerType())) \
    .withColumn("tempo_espera_medio", col("tempo_espera_medio").cast(DecimalType(10, 2))) \
    .withColumn("tempo_atendimento_medio", col("tempo_atendimento_medio").cast(DecimalType(10, 2))) \
    .withColumn("count_missing_timestamps", col("count_missing_timestamps").cast(IntegerType())) \
    .withColumn("count_reabertura", col("count_reabertura").cast(IntegerType())) \
    .withColumn("count_escalacao_incorreta", col("count_escalacao_incorreta").cast(IntegerType())) \
    .withColumn("custo_total", col("custo_total").cast(DecimalType(10, 2))) \
    .withColumn("taxa_conversao_autosservico_nivel1", col("taxa_conversao_autosservico_nivel1").cast(DecimalType(10, 2))) \
    .withColumn("taxa_conversao_nivel1_nivel2", col("taxa_conversao_nivel1_nivel2").cast(DecimalType(10, 2))) \
    .withColumn("taxa_conversao_autosservico_resolvido", col("taxa_conversao_autosservico_resolvido").cast(DecimalType(10, 2))) \
    .withColumn("pct_missing_timestamps", col("pct_missing_timestamps").cast(DecimalType(10, 2))) \
    .withColumn("taxa_reabertura", col("taxa_reabertura").cast(DecimalType(10, 2))) \
    .withColumn("pct_escalacao_incorreta", col("pct_escalacao_incorreta").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros finais: {df_funnel_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela transformada em formato Delta na camada Gold.

In [0]:
df_funnel_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_funnel)

gold_table_funnel = spark.table(tgt_table_funnel)
final_count_funnel = gold_table_funnel.count()

print(f"Salvo em: {tgt_table_funnel}")
print(f"Contagem final: {final_count_funnel:,}")

display(gold_table_funnel.limit(10))

### Relatório de Transformação

Resumo estatístico da transformação Silver para Gold.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - FUNNEL_METRICS")
print("="*80)
print(f"Registros Silver (origem chamados): {total_before_chamados:,}")
print(f"Registros Gold (destino): {final_count_funnel:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: anomalous_escalations

Identificação de chamados com escalações anômalas.
Detecção de chamados que escalaram sem passar por autosserviço.

### Filtro de Anomalias

Seleção de chamados que não passaram por autosserviço mas foram escalados.

In [0]:
tgt_table_anomalous = f"{catalog_name}.{gold_db_name}.anomalous_escalations"

df_anomalous = df_funnel.filter(
    (col("n_autosservico") == 0) & (col("nivel_atendimento") > 1)
).select(
    col("id_chamado"),
    col("id_cliente"),
    col("data"),
    col("canal"),
    col("nivel_atendimento"),
    col("n_autosservico"),
    col("n_nivel1"),
    col("n_nivel2"),
    col("motivo"),
    lit("Escalado sem autosservico").alias("tipo_anomalia")
).withColumn("processed_timestamp", current_timestamp())

df_anomalous_typed = df_anomalous \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("nivel_atendimento", col("nivel_atendimento").cast(IntegerType())) \
    .withColumn("n_autosservico", col("n_autosservico").cast(IntegerType())) \
    .withColumn("n_nivel1", col("n_nivel1").cast(IntegerType())) \
    .withColumn("n_nivel2", col("n_nivel2").cast(IntegerType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("tipo_anomalia", col("tipo_anomalia").cast(StringType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de anomalias detectadas: {df_anomalous_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de anomalias em formato Delta.

In [0]:
df_anomalous_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_anomalous)

gold_table_anomalous = spark.table(tgt_table_anomalous)
final_count_anomalous = gold_table_anomalous.count()

print(f"Salvo em: {tgt_table_anomalous}")
print(f"Contagem final: {final_count_anomalous:,}")

display(gold_table_anomalous.limit(10))

### Relatório de Transformação

Resumo estatístico de anomalias detectadas.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - ANOMALOUS_ESCALATIONS")
print("="*80)
print(f"Registros Silver (origem): {total_before_chamados:,}")
print(f"Registros Gold (destino): {final_count_anomalous:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: top_reasons_volume

Top motivos por volume de chamados.

### Agregação por Motivo

Contagem de chamados por motivo da dimensão dm_motivos.

In [0]:
tgt_table_top_volume = f"{catalog_name}.{gold_db_name}.top_reasons_volume"

df_top_volume = df_funnel.groupBy("motivo").agg(
    count("id_chamado").alias("volume_chamados")
).orderBy(col("volume_chamados").desc())

df_top_volume = df_top_volume.withColumn("processed_timestamp", current_timestamp())

df_top_volume_typed = df_top_volume \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_volume_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de top motivos por volume.

In [0]:
df_top_volume_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_volume)

gold_table_top_volume = spark.table(tgt_table_top_volume)
final_count_top_volume = gold_table_top_volume.count()

print(f"Salvo em: {tgt_table_top_volume}")
print(f"Contagem final: {final_count_top_volume:,}")

display(gold_table_top_volume.limit(10))

## Processamento: top_reasons_cost

Top motivos por custo de atendimento.

### Agregação por Motivo e Custo

Soma de custos por motivo da dimensão dm_motivos.

In [0]:
tgt_table_top_cost = f"{catalog_name}.{gold_db_name}.top_reasons_cost"

df_top_cost = df_funnel.groupBy("motivo").agg(
    spark_sum("custo").alias("custo_total"),
    count("id_chamado").alias("volume_chamados")
).orderBy(col("custo_total").desc())

df_top_cost = df_top_cost.withColumn(
    "custo_medio",
    spark_round(
        when(
            col("volume_chamados") > 0,
            col("custo_total") / col("volume_chamados")
        ).otherwise(lit(None)),
        2
    )
)

df_top_cost = df_top_cost.withColumn("processed_timestamp", current_timestamp())

df_top_cost_typed = df_top_cost \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("custo_total", spark_round(col("custo_total"), 2).cast(DecimalType(10, 2))) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("custo_medio", spark_round(col("custo_medio"), 2).cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_cost_typed.count():,}")


### Gravação na Camada Gold

Persistência da tabela de top motivos por custo.

In [0]:
df_top_cost_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_cost)

gold_table_top_cost = spark.table(tgt_table_top_cost)
final_count_top_cost = gold_table_top_cost.count()

print(f"Salvo em: {tgt_table_top_cost}")
print(f"Contagem final: {final_count_top_cost:,}")

display(gold_table_top_cost.limit(10))

## Processamento: top_reasons_low_satisfaction

Top motivos com menor satisfação do cliente.

### Leitura da Tabela de Pesquisa de Satisfação

Carregamento da pesquisa de satisfação e join com chamados.

In [0]:
src_table_satisfacao = f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao"
tgt_table_top_satisfaction = f"{catalog_name}.{gold_db_name}.top_reasons_low_satisfaction"

if not safe_table_exists(src_table_satisfacao):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_satisfacao}")

df_satisfacao_src = spark.table(src_table_satisfacao)

df_base_satisfaction = df_funnel.alias("f") \
    .join(df_satisfacao_src.alias("s"), col("f.id_chamado") == col("s.id_chamado"), "inner") \
    .select(
        col("f.id_chamado"),
        col("f.motivo"),
        col("s.nota_atendimento").alias("nota_satisfacao")
    )

print(f"Registros com pesquisa de satisfação: {df_base_satisfaction.count():,}")

### Agregação por Motivo

Cálculo de nota média por motivo da dimensão dm_motivos.

In [0]:
df_top_satisfaction = df_base_satisfaction.groupBy("motivo").agg(
    spark_round(avg("nota_satisfacao"), 2).cast(DecimalType(10, 2)).alias("nota_media"),
    count("id_chamado").alias("volume_respostas"),
    spark_sum(
        when(col("nota_satisfacao") <= 3, 1).otherwise(0)
    ).alias("count_baixa_satisfacao")
).orderBy(col("nota_media").asc())

df_top_satisfaction = df_top_satisfaction.withColumn(
    "perc_baixa_satisfacao",
    spark_round(
        when(
            col("volume_respostas") > 0,
            (col("count_baixa_satisfacao") / col("volume_respostas")) * 100
        ).otherwise(lit(0.0)),
        2
    ).cast(DecimalType(10, 2))
)

df_top_satisfaction = df_top_satisfaction.withColumn("processed_timestamp", current_timestamp())

df_top_satisfaction_typed = df_top_satisfaction \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("nota_media", col("nota_media").cast(DecimalType(10, 2))) \
    .withColumn("volume_respostas", col("volume_respostas").cast(IntegerType())) \
    .withColumn("count_baixa_satisfacao", col("count_baixa_satisfacao").cast(IntegerType())) \
    .withColumn("perc_baixa_satisfacao", col("perc_baixa_satisfacao").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_satisfaction_typed.count():,}")


### Gravação na Camada Gold

Persistência da tabela de motivos com baixa satisfação.

In [0]:
df_top_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_satisfaction)

gold_table_top_satisfaction = spark.table(tgt_table_top_satisfaction)
final_count_top_satisfaction = gold_table_top_satisfaction.count()

print(f"Salvo em: {tgt_table_top_satisfaction}")
print(f"Contagem final: {final_count_top_satisfaction:,}")

display(gold_table_top_satisfaction.limit(10))

## Processamento: satisfaction_trends

Tendências de satisfação ao longo do tempo por canal e região.

### Agregação Temporal

Cálculo de métricas de satisfação por data, canal e região.

In [0]:
tgt_table_satisfaction_trends = f"{catalog_name}.{gold_db_name}.satisfaction_trends"

df_satisfaction_full = df_funnel.alias("f") \
    .join(df_satisfacao_src.alias("s"), col("f.id_chamado") == col("s.id_chamado"), "inner") \
    .select(
        col("f.data"),
        col("f.semana"),
        col("f.ano"),
        col("f.canal"),
        col("f.regiao"),
        col("f.motivo"),
        col("s.nota_atendimento").alias("nota_satisfacao")
    )

df_satisfaction = df_satisfaction_full.groupBy("data", "semana", "ano", "canal", "regiao").agg(
    spark_round(avg("nota_satisfacao"), 2).cast(DecimalType(10, 2)).alias("nota_media"),
    count("nota_satisfacao").alias("total_respostas"),
    spark_sum(when(col("nota_satisfacao") >= 4, 1).otherwise(0)).alias("count_satisfeito"),
    spark_sum(when(col("nota_satisfacao") <= 3, 1).otherwise(0)).alias("count_insatisfeito")
)

df_satisfaction = df_satisfaction.withColumn(
    "perc_satisfeito",
    spark_round(
        when(
            col("total_respostas") > 0,
            (col("count_satisfeito") / col("total_respostas")) * 100
        ).otherwise(lit(0.0)),
        2
    ).cast(DecimalType(10, 2))
)

df_satisfaction = df_satisfaction.withColumn("processed_timestamp", current_timestamp())

df_satisfaction_typed = df_satisfaction \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("nota_media", col("nota_media").cast(DecimalType(10, 2))) \
    .withColumn("total_respostas", col("total_respostas").cast(IntegerType())) \
    .withColumn("count_satisfeito", col("count_satisfeito").cast(IntegerType())) \
    .withColumn("count_insatisfeito", col("count_insatisfeito").cast(IntegerType())) \
    .withColumn("perc_satisfeito", col("perc_satisfeito").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros: {df_satisfaction_typed.count():,}")


### Gravação na Camada Gold

Persistência da tabela de tendências de satisfação.

In [0]:
df_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_satisfaction_trends)

gold_table_satisfaction_trends = spark.table(tgt_table_satisfaction_trends)
final_count_satisfaction_trends = gold_table_satisfaction_trends.count()

print(f"Salvo em: {tgt_table_satisfaction_trends}")
print(f"Contagem final: {final_count_satisfaction_trends:,}")

display(gold_table_satisfaction_trends.limit(10))

## Processamento: client_profiles

Perfis de clientes baseados em histórico de atendimento.

### Agregação por Cliente

Cálculo de métricas por cliente.

In [0]:
tgt_table_client_profiles = f"{catalog_name}.{gold_db_name}.client_profiles"

w_motivo = Window.partitionBy("id_cliente").orderBy(col("count_motivo").desc())

df_motivo_por_cliente = df_funnel.groupBy("id_cliente", "motivo").agg(
    count("id_chamado").alias("count_motivo")
)

df_motivo_principal = df_motivo_por_cliente \
    .withColumn("rn", row_number().over(w_motivo)) \
    .filter(col("rn") == 1) \
    .select(col("id_cliente"), col("motivo").alias("motivo_principal"))

df_client_profiles = df_funnel.groupBy("id_cliente", "regiao").agg(
    count("id_chamado").alias("total_chamados"),
    spark_sum(when(col("resolvido") == "Sim", 1).otherwise(0)).alias("total_resolvidos"),
    spark_round(avg("tempo_espera_minutos"), 2).alias("tempo_espera_medio"),
    spark_round(spark_sum("custo"), 2).alias("custo_total_cliente")
)

df_client_profiles = df_client_profiles.join(df_motivo_principal, "id_cliente", "left")

df_client_profiles = df_client_profiles.withColumn(
    "taxa_resolucao",
    spark_round(
        when(
            col("total_chamados") > 0,
            (col("total_resolvidos") / col("total_chamados")) * 100
        ).otherwise(lit(0.0)),
        2
    )
)

df_client_profiles = df_client_profiles.withColumn("processed_timestamp", current_timestamp())

df_client_profiles_typed = df_client_profiles \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("total_chamados", col("total_chamados").cast(IntegerType())) \
    .withColumn("total_resolvidos", col("total_resolvidos").cast(IntegerType())) \
    .withColumn("tempo_espera_medio", spark_round(col("tempo_espera_medio"), 2).cast(DecimalType(10, 2))) \
    .withColumn("custo_total_cliente", spark_round(col("custo_total_cliente"), 2).cast(DecimalType(10, 2))) \
    .withColumn("motivo_principal", col("motivo_principal").cast(StringType())) \
    .withColumn("taxa_resolucao", spark_round(col("taxa_resolucao"), 2).cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de clientes únicos: {df_client_profiles_typed.count():,}")


### Gravação na Camada Gold

Persistência da tabela de perfis de clientes.

In [0]:
df_client_profiles_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_client_profiles)

gold_table_client_profiles = spark.table(tgt_table_client_profiles)
final_count_client_profiles = gold_table_client_profiles.count()

print(f"Salvo em: {tgt_table_client_profiles}")
print(f"Contagem final: {final_count_client_profiles:,}")

display(gold_table_client_profiles.limit(10))

### Relatório de Transformação

Resumo estatístico da transformação Silver para Gold.

In [0]:
print("="*80)
print("RELATÓRIO FINAL DE TRANSFORMAÇÃO - CAMADA GOLD")
print("="*80)
print(f"Tabelas Gold criadas: 7")
print(f"  - funnel_metrics: {final_count_funnel:,} registros")
print(f"  - anomalous_escalations: {final_count_anomalous:,} registros")
print(f"  - top_reasons_volume: {final_count_top_volume:,} registros")
print(f"  - top_reasons_cost: {final_count_top_cost:,} registros")
print(f"  - top_reasons_low_satisfaction: {final_count_top_satisfaction:,} registros")
print(f"  - satisfaction_trends: {final_count_satisfaction_trends:,} registros")
print(f"  - client_profiles: {final_count_client_profiles:,} registros")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)